In [13]:
from neo4j import GraphDatabase
import os

uri = "neo4j+s://5035e6af.databases.neo4j.io"
user = "neo4j"
password = os.getenv('NEO4J_PASSWORD')

driver = GraphDatabase.driver(uri, auth=(user, password))

In [8]:
def create_person_node(tx, name, properties):
    tx.run("CREATE (a:Person {name: $name, role: $role})", name=name, role=properties['role'])

def create_entity_node(tx, name, properties):
    tx.run("CREATE (e:Entity {name: $name})", name=name)

def create_relationship(tx, person, entity, relation, properties):
    if relation == "KNOWS":
        tx.run(f"MATCH (p:Person {{name: $person}}), (e:Entity {{name: $entity}}) "
               f"CREATE (p)-[:{relation} {{since: $since}}]->(e)", 
               person=person, entity=entity, since=properties['since'])
    else:
        tx.run(f"MATCH (p:Person {{name: $person}}), (e:Entity {{name: $entity}}) "
               f"CREATE (p)-[:{relation} {{start_date: $start_date, end_date: $end_date}}]->(e)", 
               person=person, entity=entity, start_date=properties['start_date'], end_date=properties['end_date'])

In [ ]:

with driver.session() as session:
    session.execute_write(create_person_node, "Andreotti", {"name": "Giulio Andreotti", "role": "Politician"})
    session.execute_write(create_person_node, "Moro", {"name": "Aldo Moro", "role": "Politician"})
    session.execute_write(create_person_node, "Berlinguer", {"name": "Enrico Berlinguer", "role": "Politician"})
    session.execute_write(create_person_node, "Neni", {"name": "Francesco Cossiga", "role": "Politician"})

    session.execute_write(create_entity_node, "DC", {"name": "Democrazia Cristiana"})
    session.execute_write(create_entity_node, "PC", {"name": "Partito Comunista"})
    session.execute_write(create_entity_node, "PS", {"name": "Partito Socialista"})

    session.execute_write(create_relationship, "Andreotti", "DC", "MEMBER", {"start_date": "01-01-1960", "end_date": "31-12-1992"})
    session.execute_write(create_relationship, "Moro", "DC", "MEMBER", {"start_date": "01-01-1960", "end_date": "31-12-1978"})
    session.execute_write(create_relationship, "Berlinguer", "PC", "MEMBER", {"start_date": "01-01-1963", "end_date": "31-12-1984"})
    session.execute_write(create_relationship, "Neni", "PS", "MEMBER", {"start_date": "01-01-1970", "end_date": "31-12-1987"})

    session.execute_write(create_relationship, "Andreotti", "Moro", "KNOWS", {"since": "01-01-1960"})

In [ ]:
open ~/.zshrc